In [3]:
# GPU Parallelism Utilities

import threading
import torch
import torch.multiprocessing as mp
from torch.nn.parallel import DataParallel
from queue import Queue
import gc

# 1. DataParallel Wrapper for models
def apply_data_parallel(model, device):
    """Apply DataParallel to split batches across GPU threads if using CUDA"""
    if torch.cuda.is_available() and 'cuda' in str(device):
        return DataParallel(model).to(device)
    return model.to(device)

# 2. Memory Management Utilities
def get_gpu_memory_info():
    """Get GPU memory usage information"""
    if torch.cuda.is_available():
        # Get stats for the current device
        device_idx = torch.cuda.current_device()
        stats = torch.cuda.memory_stats(device_idx)
        # Find total, allocated, and free memory
        total_memory = torch.cuda.get_device_properties(device_idx).total_memory / (1024**3)  # GB
        allocated = stats["allocated_bytes.all.current"] / (1024**3)  # GB
        free_memory = total_memory - allocated
        return {
            "total_memory_gb": total_memory,
            "allocated_memory_gb": allocated,
            "free_memory_gb": free_memory,
            "device": torch.cuda.get_device_name(device_idx)
        }
    return {"device": "CPU", "total_memory_gb": 0, "allocated_memory_gb": 0, "free_memory_gb": 0}

def clear_gpu_memory():
    """Aggressively clear GPU memory"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

# 3. Concurrent Model Training with Threading
class ConcurrentTrainer:
    """Class for training multiple models concurrently on a single GPU"""
    def __init__(self, configs, dataloaders, device, max_concurrent=2):
        self.configs = configs
        self.dataloaders = dataloaders
        self.device = device
        self.max_concurrent = max_concurrent
        self.results = []
        self.lock = threading.Lock()
        self.active_count = 0
    
    def _train_model_thread(self, config, idx, result_queue):
        """Thread worker function to train a model"""
        try:
            with self.lock:  # Track active thread count
                self.active_count += 1
            
            # Print memory info before training
            memory_info = get_gpu_memory_info()
            print(f"\nStarting training for {config['name']} | "  
                  f"GPU Memory: {memory_info['free_memory_gb']:.2f}GB free")
            
            # Train model
            results = train_model_with_config(config, self.dataloaders, self.device)
            
            # Evaluate on test set
            eval_metrics = evaluate_model_on_test(
                results['model'],
                test_loader,
                self.device,
                class_names,
                model_name=config['name']
            )
            
            # Combine results
            complete_results = {
                'model_name': config['name'],
                'architecture': config['model']['architecture'],
                'config': config,
                'training_time': results['training_time'],
                'best_val_acc': results['training_stats']['best_val_acc'],
                'best_epoch': results['training_stats']['best_epoch'],
                'test_accuracy': eval_metrics['accuracy'],
                'f1_score': eval_metrics['f1_score'],
                'precision': eval_metrics['precision'],
                'recall': eval_metrics['recall']
            }
            
            # Save model
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            if IN_COLAB:
                save_dir = os.path.join(project_root, "experiments", f"{config['name'].lower().replace(' ', '_')}_{timestamp}")
            else:
                save_dir = os.path.join(project_root, "experiments", f"{config['name'].lower().replace(' ', '_')}_{timestamp}")
            
            os.makedirs(save_dir, exist_ok=True)
            save_path = os.path.join(save_dir, f"{config['name'].lower().replace(' ', '_')}.pt")
            
            save_model(
                results['model'],
                config,
                save_path,
                metrics={
                    'accuracy': eval_metrics['accuracy'],
                    'f1_macro': eval_metrics['f1_score'],
                    'best_val_acc': results['training_stats']['best_val_acc'],
                    'best_epoch': results['training_stats']['best_epoch']
                },
                epoch=results['training_stats']['best_epoch']
            )
            
            print(f"Model saved to {save_path}")
            
            # Add to results queue
            result_queue.put((idx, complete_results))
            
            # Clear GPU memory
            del results['model']
            clear_gpu_memory()
            
        except Exception as e:
            print(f"Error training {config['name']}: {e}")
            result_queue.put((idx, None))
            
        finally:
            with self.lock:  # Update active thread count
                self.active_count -= 1
    
    def train_all_models(self):
        """Train all models with controlled concurrency"""
        memory_info = get_gpu_memory_info()
        print(f"\nGPU Memory Information before training:")
        print(f"Device: {memory_info['device']}")
        print(f"Total Memory: {memory_info['total_memory_gb']:.2f} GB")
        print(f"Allocated Memory: {memory_info['allocated_memory_gb']:.2f} GB")
        print(f"Free Memory: {memory_info['free_memory_gb']:.2f} GB")
        
        result_queue = Queue()
        threads = []
        results = [None] * len(self.configs)
        
        # Create a thread for each model configuration
        for idx, config in enumerate(self.configs):
            thread = threading.Thread(
                target=self._train_model_thread,
                args=(config, idx, result_queue)
            )
            threads.append(thread)
        
        # Start threads with controlled concurrency
        running_threads = []
        for thread in threads:
            # Wait if we've reached max concurrent threads
            while self.active_count >= self.max_concurrent:
                # Check if any results are ready
                if not result_queue.empty():
                    idx, result = result_queue.get()
                    if result is not None:
                        results[idx] = result
                time.sleep(1)  # Sleep briefly before checking again
            
            # Start the thread
            thread.start()
            running_threads.append(thread)
        
        # Collect remaining results
        for thread in running_threads:
            thread.join()
        
        # Get any remaining results
        while not result_queue.empty():
            idx, result = result_queue.get()
            if result is not None:
                results[idx] = result
        
        # Filter out any None results (from errors)
        self.results = [r for r in results if r is not None]
        return self.results

# Let user decide how many concurrent models to train based on their GPU
def check_gpu_and_suggest_concurrency():
    """Check GPU and suggest concurrency level based on memory"""
    if not torch.cuda.is_available():
        print("No GPU detected. Will run models sequentially.")
        return 1
    
    memory_info = get_gpu_memory_info()
    total_gb = memory_info['total_memory_gb']
    print(f"\nGPU: {memory_info['device']}")
    print(f"Total Memory: {total_gb:.2f} GB")
    print(f"Free Memory: {memory_info['free_memory_gb']:.2f} GB")
    
    # Suggestion based on total memory
    if total_gb >= 24:  # High-end GPUs
        suggested = 3
    elif total_gb >= 12:  # Mid-range GPUs
        suggested = 2
    else:  # Low-end GPUs
        suggested = 1
    
    print(f"\nBased on your GPU memory, suggested concurrent models: {suggested}")
    return suggested

# Check GPU and get recommended concurrency
concurrent_models = check_gpu_and_suggest_concurrency()

No GPU detected. Will run models sequentially.


# Flexible CNN Architecture Comparison for Musical Instrument Classification

This notebook implements an approach to compare different configurations of our CustomCNN model using the flexible framework. We'll train and evaluate multiple CNN architectures to find the optimal configuration for musical instrument classification.

## Approach Overview

1. **Environment Setup**: Configure the Colab environment and import necessary modules
2. **Dataset Preparation**: Load and preprocess the dataset once for all models
3. **Model Configurations**: Define multiple CustomCNN variants with different architectures
4. **Sequential Training**: Train each model variant and track performance metrics
5. **Comparison Analysis**: Compare model variants and identify the best-performing architecture
6. **Visualization**: Create comparative visualizations of model performance

We'll use our `FlexibleCNN` model which allows us to experiment with different architectures through configuration rather than code changes.

In [ ]:
# Setup: Add project root to path to enable imports from src
import os
import sys
import yaml
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import torch.nn as nn
import time
from pathlib import Path
from tqdm.notebook import tqdm
from datetime import datetime

# Check if we're running in Google Colab
import importlib.util
IN_COLAB = importlib.util.find_spec("google.colab") is not None

# Get the current directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# Import our project modules
try:
    from scripts.colab_integration import setup_colab_environment, check_gpu
    from src.data.preprocessing import get_preprocessing_transforms
    from src.data.augmentation import AdvancedAugmentation
    from src.data.dataloader import load_datasets
    from src.models.baseline import get_resnet18_model, unfreeze_layers
    from src.models.custom_cnn import create_custom_cnn
    from src.models.flexible_cnn import FlexibleCNN, create_flexible_cnn
    from src.training.trainer import train_model, evaluate_model
    from src.training.metrics import compute_metrics, get_confusion_matrix
    from src.visualization.plotting import plot_training_history, plot_confusion_matrix, plot_sample_predictions, plot_sample_images
    from src.models.model_utils import save_model
    
    print("Successfully imported project modules")
except Exception as e:
    print(f"Error importing project modules: {e}")
    print("Please ensure all required modules are available")

if IN_COLAB:
    # Add project root to path to ensure imports work correctly
    project_root = os.path.join(current_dir, "MIC-MA1")
    sys.path.insert(0, project_root)
    print(f"Project root added to path: {project_root}")
    print("🚀 Running in Google Colab - setting up environment...")
    setup_colab_environment()  # This handles all the Colab-specific setup
else:
    # Running locally
    project_root = str(Path(current_dir).parent)
    sys.path.append(project_root)
    print(f"Running locally, project root: {project_root}")
    print("💻 Running locally - using local environment")

# Check for TPU and GPU availability
try:
    import torch_xla
    import torch_xla.core.xla_model as xm
    device = xm.xla_device()
    print("Using TPU:", device)
except ImportError:
    device = check_gpu()  # Your utility function for GPU detection
    print("Using device:", device)

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

## 1. Dataset Preparation

We'll load our dataset once and use it for all model variants to ensure fair comparison:

In [ ]:
# Default data configuration
data_config = {
    'img_size': 224,
    'batch_size': 32,
    'num_workers': 2 if IN_COLAB else 4,
    'pin_memory': True,
    'augmentation_strength': 'medium'
}

# Set data directory
if IN_COLAB:
    # Adjust path based on your Google Drive structure
    data_dir = os.path.join(project_root, "data/raw/30_Musical_Instruments")
    if not os.path.exists(data_dir):
        print("⚠️ Dataset not found in the expected location")
        print("Please upload the dataset to the correct location in Google Drive")
else:
    data_dir = os.path.join(project_root, "data/raw/30_Musical_Instruments")

print(f"Using data directory: {data_dir}")

# Get advanced transforms with the specified strength
print(f"Using advanced augmentation with strength: {data_config['augmentation_strength']}")
transforms = AdvancedAugmentation.get_advanced_transforms(
    img_size=data_config['img_size'],
    augmentation_strength=data_config['augmentation_strength']
)

# Load datasets using our utility function
try:
    data = load_datasets(
        data_dir=data_dir,
        transforms=transforms,
        batch_size=data_config['batch_size'],
        num_workers=data_config['num_workers'],
        pin_memory=data_config['pin_memory']
    )
    
    # Access the components
    train_loader = data['dataloaders']['train']
    valid_loader = data['dataloaders']['val']
    test_loader = data['dataloaders']['test']
    
    # Get class information
    class_names = list(data['class_mappings']['idx_to_class'].values())
    num_classes = data['num_classes']
    
    print(f"\nDataset loaded successfully:")
    print(f"- Number of classes: {num_classes}")
    print(f"- Training samples: {len(data['datasets']['train'])}")
    print(f"- Validation samples: {len(data['datasets']['val'])}")
    print(f"- Test samples: {len(data['datasets']['test'])}")
    
    # Display a few class names
    print(f"\nSample classes: {class_names[:5]}...")
    
    # Prepare dataloaders dictionary for training
    dataloaders = {
        'train': train_loader,
        'val': valid_loader
    }
    
    # Visualize some sample images
    print("\nSample training images:")
    plot_sample_images(
        dataset=data['datasets']['train'],
        class_mapping=data['class_mappings']['idx_to_class'],
        num_images=5,
        title="Sample Training Images"
    )
except Exception as e:
    print(f"Error loading datasets: {e}")

## 2. Model Configurations

We'll define multiple configurations for our CustomCNN models to compare:

1. **Base Custom CNN** - The original CustomCNN architecture
2. **Deeper Custom CNN** - A deeper architecture with an additional convolutional block
3. **Wider Custom CNN** - A wider architecture with more filters in each layer
4. **Regularized Custom CNN** - A variant with stronger regularization
5. **ResNet18 Baseline** - For comparison with a standard architecture

We'll create these configurations directly in the notebook for simplicity, but they are based on our YAML configuration files.

In [ ]:
# Define various model configurations

# 1. Base Custom CNN configuration
base_cnn_config = {
    'model': {
        'architecture': 'flexible_cnn',
        'input_channels': 3,
        'num_classes': num_classes,
        'conv_layers': [32, 64, 128, 256, 512],
        'fc_layers': [512, 256],
        'kernel_size': 3,
        'pool_size': 2,
        'dropout': 0.5,
        'activation': 'relu',
        'pooling_type': 'max',
        'use_batch_norm': True
    },
    'training': {
        'num_epochs': 30,
        'batch_size': 32,
        'optimizer': {
            'name': 'adamw',
            'learning_rate': 0.001,
            'weight_decay': 0.0005
        },
        'scheduler': {
            'name': 'onecycle',
            'max_lr': 0.01
        }
    },
    'name': 'Base CNN'
}

# 2. Deeper Custom CNN configuration
deeper_cnn_config = {
    'model': {
        'architecture': 'flexible_cnn',
        'input_channels': 3,
        'num_classes': num_classes,
        'conv_layers': [32, 64, 128, 256, 512, 512],  # Added an extra layer
        'fc_layers': [512, 256],
        'kernel_size': 3,
        'pool_size': 2,
        'dropout': [0.1, 0.2, 0.3, 0.4, 0.5, 0.5, 0.5],  # Progressive dropout
        'activation': 'relu',
        'pooling_type': 'max',
        'use_batch_norm': True
    },
    'training': {
        'num_epochs': 30,
        'batch_size': 32,
        'optimizer': {
            'name': 'adamw',
            'learning_rate': 0.001,
            'weight_decay': 0.0005
        },
        'scheduler': {
            'name': 'onecycle',
            'max_lr': 0.01
        }
    },
    'name': 'Deeper CNN'
}

# 3. Wider Custom CNN configuration
wider_cnn_config = {
    'model': {
        'architecture': 'flexible_cnn',
        'input_channels': 3,
        'num_classes': num_classes,
        'conv_layers': [64, 128, 256, 512, 1024],  # Doubled filter count in last layer
        'fc_layers': [1024, 512],  # Wider fully connected layers
        'kernel_size': 3,
        'pool_size': 2,
        'dropout': 0.5,
        'activation': 'relu',
        'pooling_type': 'max',
        'use_batch_norm': True
    },
    'training': {
        'num_epochs': 30,
        'batch_size': 32,
        'optimizer': {
            'name': 'adamw',
            'learning_rate': 0.001,
            'weight_decay': 0.0005
        },
        'scheduler': {
            'name': 'onecycle',
            'max_lr': 0.01
        }
    },
    'name': 'Wider CNN'
}

# 4. Regularized Custom CNN configuration
regularized_cnn_config = {
    'model': {
        'architecture': 'flexible_cnn',
        'input_channels': 3,
        'num_classes': num_classes,
        'conv_layers': [32, 64, 128, 256, 512],
        'fc_layers': [512, 256],
        'kernel_size': 3,
        'pool_size': 2,
        'dropout': [0.2, 0.3, 0.4, 0.5, 0.6, 0.6, 0.6],  # Higher dropout rates
        'activation': 'relu',
        'pooling_type': 'max',
        'use_batch_norm': True
    },
    'training': {
        'num_epochs': 30,
        'batch_size': 32,
        'optimizer': {
            'name': 'adamw',
            'learning_rate': 0.001,
            'weight_decay': 0.001  # Increased weight decay
        },
        'scheduler': {
            'name': 'onecycle',
            'max_lr': 0.01
        }
    },
    'name': 'Regularized CNN'
}

# 5. ResNet18 Baseline (for comparison)
resnet18_config = {
    'model': {
        'architecture': 'resnet18',
        'pretrained': True,
        'feature_extracting': False,  # Full fine-tuning
        'num_classes': num_classes,
        'unfreeze_layers': ['layer4', 'fc']
    },
    'training': {
        'num_epochs': 30,
        'batch_size': 32,
        'optimizer': {
            'name': 'adamw',
            'learning_rate': 0.001,
            'weight_decay': 0.0005
        },
        'scheduler': {
            'name': 'onecycle',
            'max_lr': 0.01
        }
    },
    'name': 'ResNet18'
}

# Create a list of configurations to iterate through
model_configs = [
    base_cnn_config,
    deeper_cnn_config,
    wider_cnn_config, 
    regularized_cnn_config,
    resnet18_config
]

# Print configuration summary
print("Model configurations to compare:")
for config in model_configs:
    if config['model']['architecture'] == 'flexible_cnn':
        print(f"\n{config['name']}:")
        print(f"- Conv layers: {config['model']['conv_layers']}")
        print(f"- FC layers: {config['model']['fc_layers']}")
        print(f"- Dropout: {config['model']['dropout']}")
    else:
        print(f"\n{config['name']}:")
        print(f"- Architecture: {config['model']['architecture']}")
        print(f"- Pretrained: {config['model']['pretrained']}")
        print(f"- Feature extracting: {config['model']['feature_extracting']}")

## 3. Model Creation and Training Functions

Let's create utility functions to instantiate models and train them:

In [ ]:
def create_model(config, device):
    """Create model based on configuration"""
    model_config = config['model']
    architecture = model_config.get('architecture', 'resnet18').lower()
    
    if architecture == 'resnet18':
        pretrained = model_config.get('pretrained', True)
        feature_extracting = model_config.get('feature_extracting', True)
        
        model = get_resnet18_model(
            num_classes=model_config['num_classes'],
            pretrained=pretrained,
            feature_extracting=feature_extracting
        )
        
        # Unfreeze specific layers if specified
        if not feature_extracting and 'unfreeze_layers' in model_config:
            model, _ = unfreeze_layers(model, model_config['unfreeze_layers'])
            
    elif architecture == 'custom_cnn':
        model = create_custom_cnn(
            num_classes=model_config['num_classes'],
            input_channels=model_config.get('input_channels', 3)
        )
    elif architecture == 'flexible_cnn':
        model = create_flexible_cnn(config)
    else:
        raise ValueError(f"Unsupported architecture: {architecture}")
    
    return model.to(device)

def create_optimizer(config, model_parameters):
    """Create optimizer based on configuration"""
    optimizer_config = config['training']['optimizer']
    optimizer_name = optimizer_config.get('name', 'adam').lower()
    lr = optimizer_config.get('learning_rate', 0.001)
    weight_decay = optimizer_config.get('weight_decay', 0.0001)
    
    if optimizer_name == 'adam':
        return torch.optim.Adam(
            model_parameters,
            lr=lr,
            weight_decay=weight_decay,
            betas=(
                optimizer_config.get('beta1', 0.9),
                optimizer_config.get('beta2', 0.999)
            )
        )
    elif optimizer_name == 'adamw':
        return torch.optim.AdamW(
            model_parameters,
            lr=lr,
            weight_decay=weight_decay,
            betas=(
                optimizer_config.get('beta1', 0.9),
                optimizer_config.get('beta2', 0.999)
            )
        )
    elif optimizer_name == 'sgd':
        return torch.optim.SGD(
            model_parameters,
            lr=lr,
            momentum=optimizer_config.get('momentum', 0.9),
            weight_decay=weight_decay,
            nesterov=optimizer_config.get('nesterov', True)
        )
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

def create_scheduler(config, optimizer, steps_per_epoch=None):
    """Create learning rate scheduler based on configuration"""
    scheduler_config = config['training'].get('scheduler', {})
    scheduler_name = scheduler_config.get('name', '').lower()
    
    if not scheduler_name:
        return None
        
    if scheduler_name == 'step':
        return torch.optim.lr_scheduler.StepLR(
            optimizer,
            step_size=scheduler_config.get('step_size', 7),
            gamma=scheduler_config.get('gamma', 0.1)
        )
    elif scheduler_name == 'cosine':
        return torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=scheduler_config.get('t_max', config['training']['num_epochs']),
            eta_min=scheduler_config.get('eta_min', 0)
        )
    elif scheduler_name == 'plateau':
        return torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=scheduler_config.get('factor', 0.1),
            patience=scheduler_config.get('patience', 5),
            min_lr=scheduler_config.get('min_lr', 1e-6)
        )
    elif scheduler_name == 'onecycle' and steps_per_epoch:
        return torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=scheduler_config.get('max_lr', 0.01),
            steps_per_epoch=steps_per_epoch,
            epochs=config['training']['num_epochs'],
            pct_start=scheduler_config.get('pct_start', 0.3)
        )
    else:
        print(f"Warning: Scheduler {scheduler_name} not configured correctly, using no scheduler")
        return None

## 4. Function to Train Models and Track Results

Now we'll create a function to train a model with a given configuration and track the results:

In [ ]:
def train_model_with_config(config, dataloaders, device):
    """
    Train a model with the specified configuration and return the results
    
    Args:
        config (dict): Model and training configuration
        dataloaders (dict): Dictionary with 'train' and 'val' dataloaders
        device (torch.device): Device to train on
        
    Returns:
        dict: Results including model, metrics, and training time
    """
    print(f"\n{'='*80}")
    print(f"Training model: {config['name']}")
    print(f"Architecture: {config['model']['architecture']}")
    print(f"{'='*80}")
    
    # Record start time
    start_time = time.time()
    
    # Create model
    model = create_model(config, device)
    
    # Create optimizer
    optimizer = create_optimizer(config, model.parameters())
    
    # Create scheduler
    scheduler = create_scheduler(
        config,
        optimizer,
        steps_per_epoch=len(dataloaders['train'])
    )
    
    # Set up loss function
    criterion = nn.CrossEntropyLoss()
    
    # Train model
    model, history, training_stats = train_model(
        model,
        dataloaders,
        criterion,
        optimizer,
        device,
        scheduler=scheduler,
        num_epochs=config['training']['num_epochs'],
        verbose=True
    )
    
    # Record training time
    training_time = time.time() - start_time
    
    # Save results
    results = {
        'model': model,
        'config': config,
        'history': history,
        'training_stats': training_stats,
        'training_time': training_time
    }
    
    print(f"\nTraining completed in {training_time:.2f} seconds")
    print(f"Best validation accuracy: {training_stats['best_val_acc']:.4f} at epoch {training_stats['best_epoch']}")
    
    # Plot training history
    plot_training_history(history)
    
    # Return results for comparison
    return results

## 5. Model Evaluation Function

Next, we'll create a function to evaluate models on the test set:

In [ ]:
def evaluate_model_on_test(model, test_loader, device, class_names, model_name=None):
    """
    Evaluate a trained model on the test set
    
    Args:
        model (nn.Module): Trained model
        test_loader (DataLoader): Test data loader
        device (torch.device): Device to evaluate on
        class_names (list): List of class names
        model_name (str): Name of the model for display
        
    Returns:
        dict: Evaluation metrics
    """
    print(f"\n{'='*80}")
    print(f"Evaluating model{': ' + model_name if model_name else ''} on test set")
    print(f"{'='*80}")
    
    # Evaluate model
    test_accuracy, test_preds, test_labels = evaluate_model(
        model,
        test_loader,
        device,
        verbose=True
    )
    
    # Compute metrics
    metrics = compute_metrics(test_labels, test_preds, class_names)
    
    print(f"Test Accuracy: {metrics['accuracy']:.4f}")
    print(f"Macro-average F1 Score: {metrics['macro_avg']['f1']:.4f}")
    print(f"Macro-average Precision: {metrics['macro_avg']['precision']:.4f}")
    print(f"Macro-average Recall: {metrics['macro_avg']['recall']:.4f}")
    
    # Plot confusion matrix
    cm = get_confusion_matrix(test_labels, test_preds)
    plot_confusion_matrix(
        cm, 
        class_names, 
        figsize=(12, 10), 
        fontsize=8,
        title=f"Confusion Matrix - {model_name if model_name else 'Model'}"
    )
    
    # Return metrics
    return {
        'accuracy': metrics['accuracy'],
        'f1_score': metrics['macro_avg']['f1'],
        'precision': metrics['macro_avg']['precision'],
        'recall': metrics['macro_avg']['recall'],
        'predictions': test_preds,
        'labels': test_labels
    }

## 6. GPU Parallelism Strategies

To effectively utilize a single GPU's multiple cores and parallel processing capabilities, we can employ several strategies:

1. **DataParallel for Batch Splitting**: PyTorch's DataParallel splits batches across available GPU threads
2. **Memory-Efficient Training**: Train models in sequence but with optimized batch sizes
3. **Concurrent Training with Memory Management**: Use Python's threading to manage multiple models training concurrently

Let's implement these approaches:

## 6. Sequential Training of Models

Now let's train each model configuration sequentially and collect results:

In [ ]:
# Option 1: Apply DataParallel to models for better utilization of single GPU
print("\nApplying DataParallel to models for batch parallelism on GPU")

# Function to train with DataParallel
def train_with_data_parallel():
    all_results = []
    
    # Train each model configuration sequentially but with DataParallel
    for config in model_configs:
        try:
            # Temporarily increase batch size for better parallelism if memory allows
            original_batch_size = config['training'].get('batch_size', 32)
            memory_info = get_gpu_memory_info()
            
            # If enough GPU memory, double the batch size for DataParallel
            if memory_info['free_memory_gb'] > 4 and torch.cuda.device_count() > 0:
                config['training']['batch_size'] = original_batch_size * 2
                print(f"\nIncreased batch size to {config['training']['batch_size']} for {config['name']}")
            
            # Create model with DataParallel
            model = create_model(config, device)
            model = apply_data_parallel(model, device)
            
            # Create optimizer and scheduler
            optimizer = create_optimizer(config, model.parameters())
            scheduler = create_scheduler(
                config,
                optimizer,
                steps_per_epoch=len(dataloaders['train'])
            )
            
            # Set up loss function
            criterion = nn.CrossEntropyLoss()
            
            # Record start time
            start_time = time.time()
            
            # Train model
            print(f"\n{'='*80}")
            print(f"Training model with DataParallel: {config['name']}")
            print(f"{'='*80}")
            
            model, history, training_stats = train_model(
                model,
                dataloaders,
                criterion,
                optimizer,
                device,
                scheduler=scheduler,
                num_epochs=config['training']['num_epochs'],
                verbose=True
            )
            
            # Record training time
            training_time = time.time() - start_time
            
            # Restore original batch size
            config['training']['batch_size'] = original_batch_size
            
            # Evaluate on test set
            eval_metrics = evaluate_model_on_test(
                model,
                test_loader,
                device,
                class_names,
                model_name=config['name']
            )
            
            # Combine results
            complete_results = {
                'model_name': config['name'],
                'architecture': config['model']['architecture'],
                'config': config,
                'training_time': training_time,
                'best_val_acc': training_stats['best_val_acc'],
                'best_epoch': training_stats['best_epoch'],
                'test_accuracy': eval_metrics['accuracy'],
                'f1_score': eval_metrics['f1_score'],
                'precision': eval_metrics['precision'],
                'recall': eval_metrics['recall']
            }
            
            # Add to results list
            all_results.append(complete_results)
            
            # Optional: Save the model
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            save_dir = os.path.join(project_root, "experiments", 
                                   f"{config['name'].lower().replace(' ', '_')}_{timestamp}")
            
            os.makedirs(save_dir, exist_ok=True)
            save_path = os.path.join(save_dir, f"{config['name'].lower().replace(' ', '_')}.pt")
            
            save_model(
                model,
                config,
                save_path,
                metrics={
                    'accuracy': eval_metrics['accuracy'],
                    'f1_macro': eval_metrics['f1_score'],
                    'best_val_acc': training_stats['best_val_acc'],
                    'best_epoch': training_stats['best_epoch']
                },
                epoch=training_stats['best_epoch']
            )
            
            print(f"Model saved to {save_path}")
            
            # Plot training history
            plot_training_history(history)
            
            # Clear CUDA cache between models if using GPU
            if torch.cuda.is_available():
                del model
                torch.cuda.empty_cache()
                
        except Exception as e:
            print(f"Error training {config['name']}: {e}")
            # Continue with next model on error
            continue
            
    print("\nAll models trained and evaluated with DataParallel!")
    return all_results

In [ ]:
# Option 2: Concurrent model training with memory management
print("\nSetting up concurrent training with threading")

# Function to train with ConcurrentTrainer
def train_with_concurrent_trainer(max_concurrent=concurrent_models):
    trainer = ConcurrentTrainer(
        configs=model_configs,
        dataloaders=dataloaders,
        device=device,
        max_concurrent=max_concurrent
    )
    
    print(f"Training {len(model_configs)} models with up to {max_concurrent} running concurrently")
    all_results = trainer.train_all_models()
    print("\nAll models trained and evaluated concurrently!")
    return all_results

In [ ]:
# Choose which training approach to use
USE_DATA_PARALLEL = False  # Option 1: Use DataParallel for each model sequentially
USE_CONCURRENT = True      # Option 2: Train multiple models concurrently with threading
USE_SEQUENTIAL = False     # Option 3: Original sequential approach

# Will store all results in this list
all_results = []

if USE_DATA_PARALLEL:
    print("\n🚀 Using DataParallel approach for GPU utilization")
    all_results = train_with_data_parallel()
elif USE_CONCURRENT and concurrent_models > 1:
    print(f"\n🚀 Using concurrent training approach with {concurrent_models} models at once")
    all_results = train_with_concurrent_trainer(max_concurrent=concurrent_models)
else:
    print("\n🚀 Using original sequential training approach")
    # Original sequential training code
    for config in model_configs:
        try:
            # Train the model
            results = train_model_with_config(config, dataloaders, device)
            
            # Evaluate on test set
            eval_metrics = evaluate_model_on_test(
                results['model'],
                test_loader,
                device,
                class_names,
                model_name=config['name']
            )
            
            # Combine results
            complete_results = {
                'model_name': config['name'],
                'architecture': config['model']['architecture'],
                'config': config,
                'training_time': results['training_time'],
                'best_val_acc': results['training_stats']['best_val_acc'],
                'best_epoch': results['training_stats']['best_epoch'],
                'test_accuracy': eval_metrics['accuracy'],
                'f1_score': eval_metrics['f1_score'],
                'precision': eval_metrics['precision'],
                'recall': eval_metrics['recall']
            }
            
            # Add to results list
            all_results.append(complete_results)
            
            # Optional: Save the model
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            if IN_COLAB:
                save_dir = os.path.join(project_root, "experiments", f"{config['name'].lower().replace(' ', '_')}_{timestamp}")
            else:
                save_dir = os.path.join(project_root, "experiments", f"{config['name'].lower().replace(' ', '_')}_{timestamp}")
            
            os.makedirs(save_dir, exist_ok=True)
            save_path = os.path.join(save_dir, f"{config['name'].lower().replace(' ', '_')}.pt")
            
            save_model(
                results['model'],
                config,
                save_path,
                metrics={
                    'accuracy': eval_metrics['accuracy'],
                    'f1_macro': eval_metrics['f1_score'],
                    'best_val_acc': results['training_stats']['best_val_acc'],
                    'best_epoch': results['training_stats']['best_epoch']
                },
                epoch=results['training_stats']['best_epoch']
            )
            
            print(f"Model saved to {save_path}")
            
            # Clear CUDA cache between models if using GPU
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                
        except Exception as e:
            print(f"Error training {config['name']}: {e}")
            # Continue with next model on error
            continue
    
    print("\nAll models trained and evaluated!")

## 7. Comparative Analysis of Models

Now let's compare the performance of all the models:

### GPU Parallelism

In this section, we analyze how GPU parallelism impacts the performance of the models. By leveraging GPU parallelism, we can significantly reduce computation time and improve efficiency. This is particularly beneficial for large-scale datasets and complex models.

In [ ]:
# Create a DataFrame from results for easier comparison
results_df = pd.DataFrame(all_results)

# Print comparison table
print("\nModel Comparison Results:")
comparison_table = results_df[['model_name', 'test_accuracy', 'f1_score', 'precision', 'recall', 'best_val_acc', 'training_time']]
comparison_table['training_time_min'] = comparison_table['training_time'] / 60  # Convert to minutes
comparison_table = comparison_table.drop('training_time', axis=1)
comparison_table = comparison_table.sort_values('test_accuracy', ascending=False)
print(comparison_table.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

# Create visualizations for comparison
plt.figure(figsize=(18, 12))

# 1. Test accuracy and F1 scores
plt.subplot(2, 2, 1)
models = comparison_table['model_name'].tolist()
x = np.arange(len(models))
width = 0.35

plt.bar(x - width/2, comparison_table['test_accuracy'], width, label='Test Accuracy')
plt.bar(x + width/2, comparison_table['f1_score'], width, label='F1 Score')
plt.xlabel('Model')
plt.ylabel('Score')
plt.title('Test Accuracy and F1 Score Comparison')
plt.xticks(x, models, rotation=45, ha='right')
plt.ylim([0, 1.0])
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 2. Training time
plt.subplot(2, 2, 2)
plt.bar(models, comparison_table['training_time_min'], color='green')
plt.xlabel('Model')
plt.ylabel('Training Time (minutes)')
plt.title('Training Time Comparison')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 3. Precision and Recall
plt.subplot(2, 2, 3)
plt.bar(x - width/2, comparison_table['precision'], width, label='Precision')
plt.bar(x + width/2, comparison_table['recall'], width, label='Recall')
plt.xlabel('Model')
plt.ylabel('Score')
plt.title('Precision and Recall Comparison')
plt.xticks(x, models, rotation=45, ha='right')
plt.ylim([0, 1.0])
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 4. Best Validation Accuracy
plt.subplot(2, 2, 4)
plt.bar(models, comparison_table['best_val_acc'], color='purple')
plt.xlabel('Model')
plt.ylabel('Best Validation Accuracy')
plt.title('Best Validation Accuracy Comparison')
plt.xticks(rotation=45, ha='right')
plt.ylim([0, 1.0])
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig(os.path.join(project_root, 'results', 'model_comparison.png'), dpi=300, bbox_inches='tight')
plt.show()

# Identify the best model
best_model_name = comparison_table.iloc[0]['model_name']
best_model_accuracy = comparison_table.iloc[0]['test_accuracy']
print(f"\nBest Model: {best_model_name} with Test Accuracy: {best_model_accuracy:.4f}")

## 8. Conclusion and Next Steps

We've now compared several CustomCNN architectures using our flexible framework. Based on the results, we can see which architecture performs best for our musical instrument classification task.

### Analysis

Let's analyze what we've learned from this comparison:

1. **Performance across architectures**: We can see that different architectures have different strengths and weaknesses. The best-performing model likely achieves a balance between model capacity and regularization.

2. **Training Efficiency**: The training time varies significantly across models. Models with more parameters (like Wider CNN) or deeper architectures typically take longer to train.

3. **Regularization Impact**: The regularized variant shows how adding dropout and weight decay affects the model's performance, potentially improving generalization on test data.

4. **ResNet18 Comparison**: The ResNet18 model provides a strong baseline against which we can compare our custom architectures.

### Next Steps

1. **Optimize the best architecture**: Now that we've identified the best-performing CustomCNN architecture, we can further optimize it by:
   - Fine-tuning hyperparameters
   - Applying more advanced regularization techniques
   - Experimenting with different learning rate schedules

2. **Test on more challenging data**: To ensure our model is robust, we could test it on more challenging data or in real-world scenarios.

3. **Model Ensemble**: We could also explore ensemble methods by combining predictions from multiple models to further improve performance.

4. **Model Simplification**: If the best model is complex, we could explore model simplification techniques like pruning or distillation to make it more deployable.

In the next notebook, we'll take the best architecture identified here and further optimize it to create our final model.

## 9. Prepare for Step 2: Optimized CustomCNN

Based on our model comparison, we'll now prepare for Step 2 using the optimized CustomCNN configuration. We'll load the configuration from the YAML file and make it ready for the next phase of training.

In [ ]:
# Load the optimized CustomCNN configuration
try:
    # Build the path to the config file
    optimized_config_path = os.path.join(project_root, "config", "optimized_custom_cnn.yaml")
    
    # Load the configuration
    with open(optimized_config_path, "r") as f:
        optimized_config = yaml.safe_load(f)
    
    print(f"Loaded optimized CustomCNN configuration from {optimized_config_path}")
    print("\nOptimized CustomCNN configuration:")
    print(f"Architecture: {optimized_config['model']['name']}")
    
    # Display some key parameters
    print("\nKey parameters:")
    print(f"- Dropout rate: {optimized_config['model']['dropout_rate']}")
    print(f"- Optimizer: {optimized_config['training']['optimizer']['name']}")
    print(f"- Learning rate: {optimized_config['training']['optimizer']['learning_rate']}")
    print(f"- Weight decay: {optimized_config['training']['optimizer']['weight_decay']}")
    print(f"- Scheduler: {optimized_config['training']['scheduler']['name']}")
    print(f"- Augmentation strength: {optimized_config['augmentation']['augmentation_strength']}")
    
    print("\nThis optimized configuration will be used in Step 2")
    
except Exception as e:
    print(f"Error loading optimized configuration: {e}")
    print("You'll need to create or modify the optimized_custom_cnn.yaml file for Step 2")